## JuMP库的使用
1. LP，线性规划
2. QP，二次优化
3. SOCP，二阶圆锥曲线优化
3. MILP，混合整数线性规划
4. NLP，非线性规划
5. MINLP，混合整数非线性规划
6. SDP，半正定规划
7. MISDP，混合整数半正定规划

### 1. 一个基本的使用流程

In [ ]:
using JuMP
using SCS

In [5]:
model = Model(with_optimizer(SCS.Optimizer))
@variable(model, 0 <= x <= 2)       # 创建包含自身边界的变量
@variable(model, 0 <= y <= 30)
@objective(model, Max, 5x + 3 * y)
@constraint(model, con, 1x + 5y <= 3)

con : x + 5 y ≤ 3.0

In [6]:
optimize!(model)


----------------------------------------------------------------------------
	SCS v2.0.2 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2017
----------------------------------------------------------------------------
Lin-sys: sparse-indirect, nnz in A = 6, CG tol ~ 1/iter^(2.00)
eps = 1.00e-05, alpha = 1.50, max_iters = 5000, normalize = 1, scale = 1.00
acceleration_lookback = 20, rho_x = 1.00e-03
Variables n = 2, constraints m = 5
Cones:	linear vars: 5
Setup time: 5.20e-05s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 5.85e+19  1.81e+19  8.38e-01 -2.03e+21 -1.79e+20  1.63e+20  2.56e-05 
    60| 2.24e-08  1.30e-08  7.96e-08 -1.06e+01 -1.06e+01  1.86e-15  1.43e-04 
----------------------------------------------------------------------------
Status: Solved
Timing: Sol

In [8]:
# 求解器可能因为超时的原因停止，调用改函数查看终止状态
# 返回 OPTIMAL，表示结果是优的
termination_status(model)

OPTIMAL::TerminationStatusCode = 1

In [9]:
# 获取模型的求解信息
# primal-dual pair是一种常用的优化方法，对应其中的参数状况
primal_status(model)
dual_status(model)
value(x)
value(y)
dual(con)

FEASIBLE_POINT::ResultStatusCode = 1

### 2. 一个简单的LP示例

In [12]:
using JuMP, GLPK, Test

┌ Info: Precompiling GLPK [60bf3e95-4087-53dc-ae20-288a0d20c6a6]
└ @ Base loading.jl:1242


In [13]:
# 一个简单的线性规划
function example_basic(; verbose = true)
    model = Model(with_optimizer(GLPK.Optimizer))

    @variable(model, 0 <= x <= 2)
    @variable(model, 0 <= y <= 30)

    @objective(model, Max, 5x + 3y)
    @constraint(model, 1x + 5y <= 3.0)

    if verbose
        print(model)
    end

    JuMP.optimize!(model)

    obj_value = JuMP.objective_value(model)
    x_value = JuMP.value(x)
    y_value = JuMP.value(y)

    if verbose
        println("Objective value: ", obj_value)
        println("x = ", x_value)
        println("y = ", y_value)
    end

    # 这个@test以及≈的使用还是有点意思的
    @test obj_value ≈ 10.6
    @test x_value ≈ 2
    @test y_value ≈ 0.2
end

example_basic (generic function with 1 method)

In [14]:
example_basic(verbose = false)

Test Passed

### 3. 罐头食品厂问题
问题来自：Dantzig, Linear Programming and Extensions, Princeton University Press, Princeton, NJ, 1963.

说明：

In [15]:
function example_cannery(; verbose = true)
    plants = ["Seattle", "San-Diego"]
    markets = ["New-York", "Chicago", "Topeka"]

    # 两个原产地和三个目标城市
    capacity = [350, 600]
    demand = [300, 300, 300]

    # 两个原产地分别到三个城市的距离
    distance = [2.5 1.7 1.8; 2.5 1.8 1.4]

    # 运输单价
    freight = 90

    num_plants = length(plants)
    num_markets = length(markets)

    cannery = Model(with_optimizer(GLPK.Optimizer))

    # 创建变量，分别对应运输数量
    @variable(cannery, ship[1:num_plants, 1:num_markets] >= 0)

    # i表示两个原产地，j表示单个原产地到所有的城市，
    @constraint(cannery, capacity_con[i in 1:num_plants],
        sum(ship[i,j] for j in 1:num_markets) <= capacity[i]
    )

    # 运送量至少大于城市的需求
    @constraint(cannery, demand_con[j in 1:num_markets],
        sum(ship[i,j] for i in 1:num_plants) >= demand[j]
    )

    # 目标函数
    @objective(cannery, Min, sum(distance[i, j] * freight * ship[i, j]
        for i in 1:num_plants, j in 1:num_markets)
    )

    JuMP.optimize!(cannery)

    # 自定义输出结果
    if verbose
        println("RESULTS:")
        for i in 1:num_plants
            for j in 1:num_markets
                println("  $(plants[i]) $(markets[j]) = $(JuMP.value(ship[i, j]))")
            end
        end
    end

    # 测试终止条件等，这个可以
    @test JuMP.termination_status(cannery) == MOI.OPTIMAL
    @test JuMP.primal_status(cannery) == MOI.FEASIBLE_POINT
    @test JuMP.objective_value(cannery) == 151200.0
end

example_cannery (generic function with 1 method)

In [16]:
example_cannery(verbose = false)

Test Passed

### 4. 优化控制问题
问题来自：H. Maurer and H.D. Mittelman, "The non-linear beam via optimal control with bound state variables", Optimal Control Applications and Methods 12, pp. 19-31, 1991.


In [21]:
using Ipopt
function example_clnlbeam()
    N = 1000      # 总步数
    h = 1/N       
    alpha = 350
    model = Model(with_optimizer(Ipopt.Optimizer, print_level = 0))
    @variables(model, begin
        -1 <= t[1:(N + 1)] <= 1
        -0.05 <= x[1:(N + 1)] <= 0.05
        u[1:(N + 1)]
    end)
    @NLobjective(model, Min, sum(0.5 * h * (u[i + 1]^2 + u[i]^2) +
        0.5 * alpha * h * (cos(t[i + 1]) + cos(t[i])) for i in 1:N)
    )
    @NLconstraint(model, [i = 1:N],
        x[i + 1] - x[i] - 0.5 * h * (sin(t[i + 1]) + sin(t[i])) == 0
    )
    @constraint(model, [i = 1:N],
        t[i + 1] - t[i] - 0.5 * h * u[i + 1] - 0.5 * h * u[i] == 0
    )
    JuMP.optimize!(model)

    @test JuMP.termination_status(model) == MOI.LOCALLY_SOLVED
    @test JuMP.primal_status(model) == MOI.FEASIBLE_POINT
    @test JuMP.objective_value(model) ≈ 350.0
end

┌ Info: Precompiling Ipopt [b6b21f68-93f8-5de0-b562-5493be1d77c9]
└ @ Base loading.jl:1242


example_clnlbeam (generic function with 1 method)

In [22]:
example_clnlbeam()


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************



Test Passed

### 5. 一个简单的聚类

In [23]:
using LinearAlgebra

In [ ]:
using Plots
a = Any[[2.0, 2.0], [2.5, 2.1], [7.0, 7.0],
        [2.2, 2.3], [6.8, 7.0], [7.2, 7.5]]


In [ ]:
function example_cluster(; verbose = true)
    # Data points
    n = 2
    m = 6
    a = Any[[2.0, 2.0], [2.5, 2.1], [7.0, 7.0],
            [2.2, 2.3], [6.8, 7.0], [7.2, 7.5]]
    k = 2

    # Weight matrix
    W = zeros(m, m)
    for i in 1:m
        for j in i + 1:m
            W[i, j] = W[j, i] = exp(-norm(a[i] - a[j]) / 1.0)
        end
    end

    model = Model(with_optimizer(SCS.Optimizer, verbose = 0))
    # Z >= 0, PSD，半正定约束
    @variable(model, Z[1:m, 1:m], PSD)
    @constraint(model, Z .>= 0)
    # min Tr(W(I-Z))
    @objective(model, Min, tr(W * (Matrix(1.0I, m, m) - Z)))
    # Z e = e
    @constraint(model, Z * ones(m) .== ones(m))
    # Tr(Z) = k
    @constraint(model, tr(Z) == k)

    JuMP.optimize!(model)

    Z_val = JuMP.value.(Z)

    # A simple rounding scheme
    which_cluster = zeros(Int, m)
    num_clusters = 0
    for i in 1:m
        Z_val[i, i] <= 1e-6 && continue
        if which_cluster[i] == 0
            num_clusters += 1
            which_cluster[i] = num_clusters
            for j in i + 1:m
                if norm(Z_val[i, j] - Z_val[i, i]) <= 1e-6
                    which_cluster[j] = num_clusters
                end
            end
        end
    end

    @test which_cluster == [1, 1, 2, 1, 2, 2]

    if verbose
        # Print results
        for cluster in 1:k
            println("Cluster $cluster")
            for i in 1:m
                if which_cluster[i] == cluster
                    println(a[i])
                end
            end
        end
    end
end